## **Mga Halimbawa: Multi-AI Agents para sa Pag-book ng hotel**

Sa mabilis na takbo ng mundo ngayon, ang pagpaplano ng isang business trip ay hindi lamang basta pag-book ng flight at kwarto sa hotel. Nangangailangan ito ng antas ng koordinasyon at kahusayan na maaaring maging hamon na makamit. Dito pumapasok ang Multi-AI Agents, na nagre-rebolusyon sa paraan ng pamamahala ng ating mga pangangailangan sa paglalakbay.

Isipin na may isang koponan ng mga intelihenteng ahente na kayang tumulong sa bawat aspeto ng iyong biyahe nang may katumpakan at ginhawa. Sa aming advanced na teknolohiyang AI, nakagawa kami ng mga espesyal na ahente para sa pag-book ng mga serbisyo at pag-aayos ng itineraryo, na tinitiyak ang isang maayos at walang stress na karanasan sa paglalakbay.

Ito ay isang pangunahing senaryo. Sa pagpaplano ng isang business trip, kailangan nating kumonsulta sa isang business travel agent upang makakuha ng impormasyon tungkol sa tiket sa eroplano, impormasyon sa hotel, atbp. Sa pamamagitan ng AI Agents, maaari tayong bumuo ng mga ahente para sa pag-book ng mga serbisyo at mga ahente para sa pag-aayos ng itineraryo upang magtulungan at mapabuti ang antas ng katalinuhan.


# I-initialize ang Azure AI Agent Service at kunin ang impormasyon ng konfigurasyon mula sa **.env**

### **.env** 

Gumawa ng isang .env file

**.env** ay naglalaman ng connection string ng Azure AI Agent Service, ang modelong ginagamit ng AOAI, at ang katugmang Google API Search service API, ENDPOINT, atbp.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Pangalan ng Iyong Azure AI Agent Service Model Deployment"

[**NOTE**] Kailangan mo ng modelong may 100,000 Rate Limit (Tokens kada minuto) Rate Limit na 600 (Request kada minuto)

  Maaari kang kumuha ng model sa Microsoft Foundry - Model at Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Connection String ng Iyong Azure AI Agent Service Project"

  Maaari mong makuha ang project connection string sa overview ng iyong proyekto sa AI ​​Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Iyong SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Iyong SERPAPI Search Endpoint"

Para makuha ang Model Deployment Name at Project Connection String ng Azure AI Agent Service, kailangan mong gumawa ng Azure AI Agent Service. Inirerekomenda na gamitin ang [template na ito](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) upang gawin ito nang direkta （***Note:*** Ang Azure AI Agent Service ay kasalukuyang naka-set sa isang limitadong rehiyon. Inirerekomenda na sumangguni ka sa [link na ito](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) upang i-set ang rehiyon)

Kailangan ng Agent na ma-access ang SERPAPI. Inirerekomenda na magparehistro gamit ang [link na ito](https://serpapi.com/searches). Pagkatapos magparehistro, makakakuha ka ng natatanging API KEY at ENDPOINT


# Mag-login sa Azure 

Kailangan mo na ngayong mag-login sa Azure Buksan ang terminal sa VScode at patakbuhin ang utos na `az login`


# Setup 

Upang patakbuhin ang notebook na ito, kakailanganin mong i-install ang mga sumusunod na library. Narito ang listahan ng mga kinakailangang library at ang kaukulang mga utos para sa pip install:

azure-identity: Para sa Azure authentication.
requests: Para sa paggawa ng mga HTTP requests.
semantic-kernel: Para sa semantic kernel framework (ipagpalagay na ito ay isang custom o partikular na library, maaaring kailanganin mo itong i-install mula sa isang partikular na source o repository).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Paliwanag: 
import asyncio: Ini-import nito ang asyncio na module, na nagbibigay ng suporta para sa asynchronous na programming sa Python. Pinapayagan ka nitong magsulat ng sabayang code gamit ang async at await na syntax.
from typing import Annotated: Ini-import nito ang Annotated na uri mula sa typing module. Ginagamit ang Annotated para magdagdag ng metadata sa mga type hints, na maaaring maging kapaki-pakinabang para sa iba't ibang layunin tulad ng pag-validate, dokumentasyon, o tooling


In [ ]:
import asyncio,os
from typing import Annotated

# Paliwanag:
Sa pamamagitan ng paggamit ng from dotenv import load_dotenv at load_dotenv(), madali mong mapamamahalaan ang mga setting ng configuration at sensitibong impormasyon (tulad ng mga API key at mga URL ng database) sa isang .env na file, na naghihiwalay sa mga ito mula sa iyong source code at ginagawa ang iyong aplikasyon na mas ligtas at mas madali i-configure.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Paliwanag:

Import Statement: from azure.identity.aio import DefaultAzureCredential: Ini-import nito ang DefaultAzureCredential class mula sa azure.identity.aio module. Ang aio na bahagi ng pangalan ng module ay nagpapahiwatig na ito ay dinisenyo para sa asynchronous na mga operasyon.

Layunin ng DefaultAzureCredential: Ang DefaultAzureCredential class ay bahagi ng Azure SDK para sa Python. Nagbibigay ito ng default na paraan para ma-authenticate sa mga Azure services. Sinusubukan nitong mag-authenticate gamit ang maraming pamamaraan sa isang partikular na ayos, tulad ng mga environment variables, managed identity, at Azure CLI credentials.

Asynchronous na mga Operasyon: Ipinapahiwatig ng aio module na sinusuportahan ng DefaultAzureCredential class ang asynchronous na mga operasyon. Ibig sabihin, maaari mo itong gamitin kasama ang asyncio upang magsagawa ng non-blocking na mga kahilingan sa authentication.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Paliwanag:
Nag-iimport ng iba't ibang mga module at klase mula sa semantic_kernel package. Narito ang paliwanag ng bawat import:

AgentGroupChat mula sa semantic_kernel.agents: Ang klaseng ito ay humahawak ng mga functionality na may kinalaman sa group chat para sa mga AI agent. AzureAIAgent at AzureAIAgentSettings mula sa semantic_kernel.agents.azure_ai

AzureAIAgent: Ang klaseng ito ay ginagamit upang gumawa at pamahalaan ang mga AI agent na gumagamit ng Azure AI services.

AzureAIAgentSettings: Ang klaseng ito ay ginagamit upang i-configure ang mga setting para sa AzureAIAgent. TerminationStrategy mula sa semantic_kernel.agents.strategies.termination.termination_strategy:

Ang klaseng ito ay nagtatakda ng mga estratehiya para sa pag-terminate ng pagpapatupad ng mga AI agent sa ilalim ng ilang mga kondisyon. ChatMessageContent mula sa semantic_kernel.contents.chat_message_content:

Ang klaseng ito ay ginagamit upang pangasiwaan ang nilalaman ng mga chat message.  
AuthorRole mula sa semantic_kernel.contents.utils.author_role:

Ang klaseng ito ay nagtatakda ng iba't ibang mga papel para sa mga author sa konteksto ng mga chat message.

kernel_function mula sa semantic_kernel.functions.kernel_function_decorator: Ang decorator na ito ay ginagamit upang tukuyin ang mga kernel function, na mga function na maaaring patakbuhin sa loob ng semantic kernel framework.  
Ang mga import na ito ay naghahanda ng mga kinakailangang sangkap para sa paggawa at pamamahala ng mga AI agent na maaaring makipag-ugnayan sa isang group chat na kapaligiran, posibleng para sa mga gawain tulad ng pag-book ng mga hotel o mga katulad na aktibidad.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Paliwanag:
Susunod, ini-import natin ang klase na CodeInterpreterTool mula sa azure.ai.projects.models module.

CodeInterpreterTool: Bahagi ang klaseng ito ng Azure AI SDK at ginagamit upang bigyang-kahulugan at isagawa ang code sa loob ng konteksto ng mga proyekto sa AI. Nagbibigay ito ng mga kakayahan para sa pagpapatakbo ng mga piraso ng code, pagsusuri ng code, o pagsasama ng pagpapatupad ng code sa loob ng mga AI workflow.
Itong import ay naghahanda ng kinakailangang bahagi para magamit ang CodeInterpreterTool sa iyong proyekto, na maaaring maging kapaki-pakinabang para sa mga gawain na kinasasangkutan ng pagbibigay-kahulugan at pagsasagawa ng code nang dinamiko.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explanation: 
Ang klaseng ApprovalTerminationStrategy ay nagbibigay ng isang partikular na estratehiya para sa pagtatapos ng operasyon ng isang AI agent. Magtatapos ang agent kung ang huling mensahe sa kasaysayan ng kanyang interaksyon ay naglalaman ng salitang "saved". Ito ay maaaring maging kapaki-pakinabang sa mga senaryong ang gawain ng agent ay itinuturing na tapos na kapag ito ay nakatanggap ng kumpirmasyon na ang isang bagay ay "saved". Tukuyin ang paraan ng interaksyon. Matapos masave ang plano ng reserbasyon, maaari na itong ihinto kapag natanggap ang signal na saved


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Paliwanag:

Ang linya ng code ay nag-iinitialize ng isang AzureAIAgentSettings na bagay gamit ang default o paunang mga setting sa pamamagitan ng pagtawag sa create() na pamamaraan. Ang settings na bagay na ito (ai_agent_settings) ay maaaring gamitin upang i-configure at pamahalaan ang isang AzureAIAgent na instansya.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Paliwanag:
Sa pamamagitan ng pag-import ng requests library, madali kang makakagawa ng mga HTTP request at makipag-ugnayan sa mga web services sa iyong Python code.


In [ ]:
import requests

# Paliwanag:
Ito ay isang variable na nag-iimbak ng API key para ma-access ang isang SERP (Search Engine Results Page) API service. Ang API key ay isang natatanging identifier na ginagamit upang patunayan ang mga kahilingan na nauugnay sa iyong account.

'GOOGLE_SEARCH_API_KEY': Ito ay isang placeholder string. Kailangan mong palitan ang ''GOOGLE_SEARCH_API_KEY' ng iyong aktwal na SERP API key.

Layunin: Ang layunin ng linyang ito ay iimbak ang API key sa isang variable upang magamit ito sa pagpapatunay ng mga kahilingan sa SERP API service. Kinakailangan ang API key upang ma-access ang serbisyo at magsagawa ng mga paghahanap.

Paano Makakuha ng SERP API Key: Upang makakuha ng SERP API key, sundin ang mga pangkalahatang hakbang na ito sa https://serpapi.com (ang mga eksaktong hakbang ay maaaring magbago depende sa partikular na SERP API service na iyong ginagamit):

Pumili ng SERP API Service: May ilang mga SERP API service na available, tulad ng SerpAPI, Google Custom Search JSON API, at iba pa. Piliin ang pinakaangkop sa iyong mga pangangailangan.

Mag-sign Up para sa Account:

Pumunta sa website ng napiling SERP API service https://www.serpapi.com at mag-sign up para sa isang account. Maaaring kailangan mong magbigay ng ilang pangunahing impormasyon at i-verify ang iyong email address.

Gumawa ng API Key:

Pagkatapos mag-sign up, mag-log in sa iyong account at pumunta sa seksyon ng API o dashboard. Hanapin ang opsyon upang gumawa o mag-generate ng bagong API key.
Kopyahin ang API Key:

Kapag nagawa na ang API key, kopyahin ito. Ang key na ito ay gagamitin upang patunayan ang iyong mga kahilingan sa SERP API service.
Palitan ang Placeholder:

Palitan ang placeholder sa iyong .env file


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Paliwanag:
Ang klaseng BookingPlugin ay nagbibigay ng mga pamamaraan para sa pagrereserba ng mga hotel at flights gamit ang Serpapi.com Google Search API. Ito ay bumubuo ng kinakailangang mga parameter, nagpapadala ng mga kahilingan sa API, at pinoproseso ang mga tugon upang ibalik ang may kaugnayang impormasyon sa pagrereserba. Ang API key (SERPAPI_SEARCH_API_KEY) at endpoint (SERPAPI_SEARCH_ENDPOINT) ay ginagamit upang mag-autenticate at magpadala ng mga kahilingan sa Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Paliwanag:
Ang klase na SavePlugin ay nagbibigay ng isang pamamaraan na saving_plan upang i-save ang mga plano ng biyahe gamit ang mga serbisyo ng Azure AI. Nagse-setup ito ng mga kredensyal ng Azure, lumilikha ng isang AI agent, pinoproseso ang mga input ng user upang makabuo at mai-save ang nilalaman ng plano ng biyahe, at hinahandle ang pag-save ng file at mga operasyon ng paglilinis. Ang pamamaraan ay nagbabalik ng "Saved" kapag matagumpay na natapos.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Paliwanag:
Itinakda ng code na ito ang mga Azure AI agent upang hawakan ang pag-book ng mga flight at hotel, at pag-save ng mga plano ng biyahe batay sa mga input ng user. Ginagamit nito ang mga kredensyal ng Azure upang likhain at i-configure ang mga agent, pinoproseso ang mga input ng user sa pamamagitan ng isang group chat, at tinitiyak ang wastong paglilinis matapos makumpleto ang mga gawain. Ginagamit ng mga agent ang mga partikular na plugin (BookingPlugin at SavePlugin) upang isagawa ang kani-kanilang mga gawain.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Pagtatapat**:
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagamat aming pinagsisikapang maging tumpak, mangyaring tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o di-katumpakan. Ang orihinal na dokumento sa kanyang sariling wika ang dapat ituring na opisyal na sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na human translation. Hindi kami mananagot sa anumang hindi pagkakaintindihan o maling pagpapakahulugan na nagmula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
